In [1]:
print("llm 调用tools,让llm和外界交流")

llm 调用tools,让llm和外界交流


In [3]:
# npm install
# 发起请求的库
!pip install requests

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import requests
# python 类型约定
# js 弱类型语言
# ->str 返回值的类型

# 定义函数get_weather，接收字符串类型的location（城市 / 地区）参数，返回天气信息的字符串结果。
def get_weather(location:str) -> str:
    # 接口地址
    url="https://api.seniverse.com/v3/weather/now.json"
    #请求参数
    params={ 
        "key":"S-mAe9LCYKjWDc37X",
        "location":location,
        "language":"zh-Hans"
    }
    try:
        resp=requests.get(url,params=params,timeout=10)# 发送GET请求，设置超时时间为10秒
        data=resp.json()# 解析响应内容为JSON格式 
        print(data)# 打印响应数据，方便调试
        if "results" in data:
            r=data["results"][0]# 从响应数据中提取第一个结果
            city=r["location"]["name"]# 提取城市名称
            # 当前天气对象
            now=r["now"]
            text=now["text"]
            temp=now["temperature"]
            # python 擅长机器学习 和字符处理
            return f"{city}当前天气：{text},气温{temp}度"
        else:
            return "查询失败"
    except Exception as e:
        return f"异常：{e}"
print(get_weather("抚州"))

{'results': [{'location': {'id': 'WSFXR95RZD21', 'name': '抚州', 'country': 'CN', 'path': '抚州,抚州,江西,中国', 'timezone': 'Asia/Shanghai', 'timezone_offset': '+08:00'}, 'now': {'text': '多云', 'code': '4', 'temperature': '17'}, 'last_update': '2025-12-01T21:00:21+08:00'}]}
抚州当前天气：多云,气温17度


In [21]:
# 用户user 向llm提问 自然语言
# 实时性的 工具类的 llm 无法回答
# llm和原有的互联网文明桥接起来？ 智能互联网来了
# tools 来自openai 接口定义
from openai import OpenAI

client = OpenAI(
    api_key='sk-676db137e5804e5db22375c19a69d580',
    base_url='https://api.deepseek.com/v1'
)

In [20]:
# openai 接口定义
tools=[
    # JSON 定义
    {
        # 一个工具就是一个函数
        "type":"function",
        "function":{
            "name":"get_weather0",
            "description":"获取指定城市的当前天气",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"城市名称，如'北京'"
                    }
                },
                "required":["location"]
            }
        }
    }
    
]

In [ ]:
import json
messages = [{"role": "user", "content": "北京天气怎么样"}]
response = client.chat.completions.create(
    model = 'deepseek-reasoner',
    messages = messages,
    tools = tools,
    tool_choice = "auto",
    # 生成的随机度
    temperature = 0.3
)
response_message = response.choices[0].message
# 多轮会话
print(response_message)
# tool_calls ChatCompletionMessageFunctionToolCall
messages.append(response_message)
if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        # json 字符串变成json 对象
        function_args = json.loads(tool_call.function.arguments)
        if function_name == "get_weather":
            function_response = get_weather(function_args["location"])
        else:
            function_response = "未知工具"
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
        })
else:
    print(response_message.content)

final_response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    temperature=0.3
)
print(final_response.choices[0].message.content)

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_00_Hd2ycawFBVyE2mnsxYkhWy3V', function=Function(arguments='{"location": "北京"}', name='get_weather0'), type='function', index=0)], reasoning_content='用户询问北京的天气情况。我需要调用获取天气的工具来查询北京的天气。参数是location，设置为"北京"。那么，我来调用这个函数。')
由于我目前无法直接获取实时天气数据，以下是一些关于北京天气的通用信息和查询建议：

---

### **北京当前季节天气特点（5月中旬）**
- **气温**：通常在 **15°C ~ 28°C** 之间，白天温暖，早晚较凉。
- **天气**：春季末尾，偶有沙尘或阵雨，总体以晴到多云为主。
- **建议**：出门可备薄外套，注意防晒，关注是否有大风或降雨预警。

---

### **推荐查询方式**
1. **天气预报App**：  
   - 中国天气网（www.weather.com.cn）
   - 墨迹天气、彩云天气等第三方应用。
2. **搜索引擎**：  
   在百度、谷歌搜索“北京天气”，会显示实时温度、湿度、风速等。
3. **手机自带天气工具**：  
   大多数手机系统内置天气功能，可直接添加“北京”查看。

---

### **注意事项**
- 北京春季温差较大，建议“洋葱式穿搭”，方便随时调整。
- 若遇雾霾或沙尘天气，敏感人群需佩戴口罩。

如果需要更详细的天气信息（如每小时预报、空气质量等），可以尝试以上方式查询哦！ 🌤️
